# ECON 514 Julia Tutorial

Couple of things before we start. If this kind of coding is new to you (i.e. not straight up using one line from packages like Stata or some of R), in my experience I found several things quite useful.

- Try to change my code and see what works or what breaks. Does it do what you intend it to?
- If there is a block of code that does not run or is not giving what you expected, maybe use @show to check out what specific steps are doing. You can also do this if there are lines for which you are not sure what they are doing.
- A significant part of knowing how to code is knowing how to google.
- There is a ton of Julia documentation available online. They are mostly readable, so give it a go if you are stuck.

This tutorial is not an exhaustive guide to Julia. You are not meant to be able to right away solve problems. This tutorial is meant to give you some idea of

- what a function is
- how they fit together to do stuff
- solve a system of equations in SymPy
- pickup some syntax(?)

We are going to do this by solving a game from Running Man, a Korean gameshow.

### Problem (Copied from Mike's message on Discord)

There are 8 rooms and 8 contestants.  Each person must choose to go to one of the rooms (assume the choice is once for all and each person decides simultaneously).  Once everyone is in their room, a prize in each room is split among all the players in that room.  The prize is room 1 is 40K (won I assume), the prize in rooms 2,3 and 4 is 50K won, the prize in rooms 5,6 and 7 is 70K won, while the prize in room 8 is 100K won. So if all 8 people go to room 8, each of them will get 100/8 K won. We want to

i) Find a pure strategy equilibrium

ii) Find the unique symmetric mixed equilibrium.


We start by defining some parameters.

In [1]:
valuations = [40,50,50,50,70,70,70,100] 
options = 1:1:8

# These are arrays/vetors. Note that they have square brackets. If we use normal brackets, 
# they become a list of objects ("tuples"). The list can consist of anything, for example,

tup = (1,"Wednesday 8:30 class...",[1 3 2;1 2 3])

# On one hand they do have something in common. To access the second element of say valuations, we can do 
@show valuations[2]
@show tup[2]

valuations[2] = 50
tup[2] = "Wednesday 8:30 class..."


"Wednesday 8:30 class..."

In [6]:
a = (1,2,3)
typeof(a)
b = (4,5,6)
a'*b

LoadError: MethodError: no method matching adjoint(::Tuple{Int64, Int64, Int64})
[0mClosest candidates are:
[0m  adjoint([91m::LinearAlgebra.LQ[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\lq.jl:123
[0m  adjoint([91m::LinearAlgebra.LU[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\lu.jl:75
[0m  adjoint([91m::LinearAlgebra.Rotation[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\givens.jl:58
[0m  ...

At other times they have to be handled quite differently. For example, the default operation for array is linear algebra. In R, when you have an array and you do something like valuations^2, you will square each element. Here, it will not make sense. Instead, we have to put a dot to say that we want to apply a function element-by-element. 

This will NOT run

       valuations^2

In [29]:
# Element by Element
@show valuations.^2

# v' means v transpose. Therefore this is the dot product with itself
@show valuations' * valuations

valuations .^ 2 = [1600, 2500, 2500, 2500, 4900, 4900, 4900, 10000]
valuations' * valuations = 33800


33800

In general, stuff like multiplication and addition with tuples will not work. Functions can work depending on how they are written. We will see an example of this.

In [3]:
function room_population(s)
    pop = zeros(8)
    for i =1:8
        pop[i] = sum(s.==i)
    end
    return pop
end



function purepayoff(s1,s2,s3,s4,s5,s6,s7,s8) # Function that takes in 8 numbers, each representing a player's room choice and return payoffs. 
    s = [s1,s2,s3,s4,s5,s6,s7,s8] 
    pop = room_population()   
    payoff = zeros(8,8)

    for i =1:8
        ni = sum(s.==i)
        payoff[:,i] = valuations[i]/ni .* (s.==i)
    end

    payoff = sum(payoff,dims = 2)
    return payoff[:,1]
end

function purepayoff(s) # Function that takes in an 8 x 1 vector of room choices and return payoff.
    s1,s2,s3,s4,s5,s6,s7,s8 = s
    purepayoff(s1,s2,s3,s4,s5,s6,s7,s8) 
end

function purepayoff(si,s_noti) 
    s = vcat(si,s_noti)
    return purepayoff(s)
end

function BR(s_noti) 
    f(x) = purepayoff(x,s_noti)[1]
    rooms = 1:1:8
    payoffs = f.(rooms)
    br = findmax(payoffs)[2]
    br_payoff = findmax(payoffs)[1]
    return br,br_payoff
end


function is_BR(si,s_noti)
    br,br_payoff = BR(s_noti)
    check = (purepayoff(si,s_noti)[1] == br_payoff)
    return check, br_payoff
end

function is_eqm(s)
    check = zeros(8)
    for i = 1:8
        s_i = s[i]
        s_noti = s[1:end .!=i]
        check[i] = is_BR(s_i,s_noti)[1]
    end
    return (minimum(check)==1)
end

is_eqm([8,2,3,4,5,6,7,8])


true

In [2]:
function brute_search(S;k=10000)
    count = 0
    found = false
    for i in eachindex(S)  
                cs = S[i]       
                check = is_eqm(cs)
                count = count+1

                if check == true
                    return cs
                end

    end  
    println("Did not find any eqm.")
end

"""
We can ues this to search over alll possible combinations of strategies. We can set
   full_S = [[x1,x2,x3,x4,x5,x6,x7,x8] for x1 in 1:8 for x2 in 1:8 for x3 in 1:8 for x4 in 1:8 for x5 in 1:8 for x6 in 1:8 for x7 in 1:8 for x8 in 1:8]

Then run
  search(full_S)

But this will take a very long time.
"""

function gen_candidates(n)
    candidates = [rand(1:8,8)]
    count = 1
    for i=2:n
        candidates = vcat(candidates,[rand(1:8,8)])
    end
    return candidates
end

n = 1000

candidates = gen_candidates(n);

brute_search(candidates;k=10000)


Did not find any eqm.


The probability of successfully finding a NE with n

In [3]:
p = (factorial(8)/2)/(8^8)
(1-p)^n

0.3004865831455566

Alternatively, we can start with some strategies s0. We let each player deviate one by one until nobody wants to move.

In [40]:
function player_deviation(s,k)
    si = s[k]
    s_noti = s[1:end .!=k]

    check = is_BR(si,s_noti)[1]

    if check == true
        s[k] = si
        move = false
    else
        s[k] = BR(s_noti)[1]
        move = true
    end

    return s,move
end

function greedy_move(s0)
    stable = false
    count = 0
    while stable == false
        moves = ones(8)

        for i = 1:8
            s,moves[i] = player_deviation(s0,i)
            s0 = s
        end

        stable = (maximum(moves)==0)
        count = count+1
    end
    return s0,count
end

#s0 = [8,8,8,8,8,8,8,8]
s0 = [1,1,1,1,1,1,1,1]
        
sln = greedy_move(s0)[1]
purepayoff(sln)

        

8-element Vector{Float64}:
 50.0
 70.0
 70.0
 70.0
 50.0
 50.0
 50.0
 50.0

To fixed symmetric mixed equilibrium, suppose the probabilites are denoted 

$$
p = (p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8)
$$

In the set of i such that $p_i>0$, it has to be that any two actions in that set give the same expected payoff when others are playing mixed strategy p. Now, the expected payoff from choosing room i is

$$
   \overset{7}{\underset{j=0}{\sum}} \frac{v_i}{1+j}\times C^7_j (1-p_i)^{7-j}p_i^j
$$

Therefore, if the equilibrium has m non-zero probabilities, then we have m-1 equalities. Since the probabilites have to sum to 1, we have m-1 unknowns, hence there exists a unique solution. For this to be a mixed equilibrium, we also require that the expected payoff above is greater than playing actions that have probabiity 0.

In [36]:
using SymPy

v,p,p1,p2,p3,p4,p5,p6,p7,p8 = symbols("v,p,p1 p2 p3 p4 p5 p6 p7 p8")

EU = sum( v/(1+j)* binomial(7,j)* (1-p)^(7-j)* p^j for j in 0:7);

EU1 = EU(p=>p1,v=>valuations[1])
EU2 = EU(p=>p2,v=>valuations[2])
EU3 = EU(p=>p3,v=>valuations[3])
EU4 = EU(p=>p4,v=>valuations[4])
EU5 = EU(p=>p5,v=>valuations[5])
EU6 = EU(p=>p6,v=>valuations[6])
EU7 = EU(p=>p7,v=>valuations[7])
EU8 = EU(p=>p8,v=>valuations[8])

solution = sympy.nsolve((EU2-EU1,EU2-EU3, EU2-EU4, EU2-EU5, EU2-EU6,EU2-EU7,EU2-EU8,1-p1-p2-p3-p4-p5-p6-p7-p8),(p1,p2,p3,p4,p5,p6,p7,p8),(1,1,1,1,1,1,1,1))

8×1 Matrix{Sym}:
 0.0005900395709166406840137419570967888870576042685833651064880671073501502121675
   0.06553637288318371000730577629947820555476889335433512302866178831665458158983
   0.06553637288318371000730577629947820555476889335433512302866178831665458158983
   0.06553637288318371000730577629947820555476889335433512302866178831665458158983
    0.1696779680897165722324230717817383914913708924507470226023318700539585865871
    0.1696779680897165722324230717817383914913708924507470226023318700539585865871
    0.1696779680897165722324230717817383914913708924507470226023318700539585865871
    0.2937669375103825125967997137992534199745230383161701980005309577808103452570

Let's look at the expected payoff.

In [37]:
@show EU1(solution[1])
@show EU2(solution[2])
@show EU3(solution[3])
@show EU4(solution[4])
@show EU5(solution[5])
@show EU6(solution[6])
@show EU7(solution[7])
@show EU8(solution[8])

EU1(solution[1]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU2(solution[2]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU3(solution[3]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU4(solution[4]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU5(solution[5]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU6(solution[6]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU7(solution[7]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673
EU8(solution[8]) = 39.91749186928315296783317169561232120969394345019627455584857319182604343673


39.91749186928315296783317169561232120969394345019627455584857319182604343673

Suppose we want to see if there is an equilibrium where players only mix between room 2-8. 

In [38]:
solution = sympy.nsolve((EU2-EU3, EU2-EU4, EU2-EU5, EU2-EU6,EU2-EU7,EU2-EU8,1-p2-p3-p4-p5-p6-p7-p8),(p2,p3,p4,p5,p6,p7,p8),(1,1,1,1,1,1,1))

@show EU2(solution[1])
@show EU3(solution[2])
@show EU4(solution[3])
@show EU5(solution[4])
@show EU6(solution[5])
@show EU7(solution[6])
@show EU8(solution[7])

@show EU1(0)

EU2(solution[1]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU3(solution[2]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU4(solution[3]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU5(solution[4]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU6(solution[5]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU7(solution[6]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU8(solution[7]) = 39.90698417785192965555710132508902709270987682969983827549323312327389406646
EU1(0) = 40


40

The expected payoff by deviating to room 1 is 40, which is higher than any actions we are mixing over. So this is not an equilibrium.